
# Conversation Management & Classification using Groq API (OpenAI-compatible)



## Setup instructions

1. This notebook uses only standard Python packages and `requests`. For JSON schema validation we use `jsonschema` (installed below).
2. **DO NOT** store API keys in the notebook. Instead, in Colab set them via:
```python
import os
os.environ['GROQ_API_KEY'] = "your_groq_api_key_here"
# or os.environ['OPENAI_API_KEY'] = "..." if using OpenAI-compatible variable names
```
3. The notebook has `MOCK_MODE = True` by default so you can run and verify outputs without any external calls.
4. To run real requests against Groq/OpenAI-compatible endpoints, set `MOCK_MODE = False` and set environment variables as above.


In [26]:
!pip install --quiet jsonschema

In [27]:
!pip install openai

In [33]:
from dotenv import load_dotenv
import os
import openai

# Load environment variables from .env file
load_dotenv()

# Get API key safely
openai.api_key = os.getenv("GROQ_API_KEY")
openai.api_base = "https://api.groq.com/v1"


In [34]:
import os

print("Groq API key set:", bool(os.environ.get("GROQ_API_KEY")))


Groq API key set: True


In [35]:
pip install groq


In [37]:
from groq import Groq
import re
from typing import List, Dict
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize Groq client with key from environment variable
client = Groq(api_key=os.getenv("GROQ_API_KEY"))


class ConversationManager:
    def __init__(self, summarize_after_k_runs: int = 3):
        self.history: List[Dict[str, str]] = []
        self.run_count = 0
        self.summarize_after_k_runs = summarize_after_k_runs

    def add_message(self, role: str, text: str):
        assert role in ('user', 'assistant'), "role must be 'user' or 'assistant'"
        self.history.append({'role': role, 'text': text})
        self.run_count += 1

        if self.summarize_after_k_runs > 0 and (self.run_count % self.summarize_after_k_runs) == 0:
            summary = self.summarize_history()
            self.history = [{'role': 'assistant', 'text': f'Conversation summary: {summary}'}]

    def summarize_history(self) -> str:
        if not self.history:
            return ""
        messages = [{"role": "system", "content": "Summarize the conversation concisely."}] + \
                   [{"role": m["role"], "content": m["text"]} for m in self.history]
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=messages
        )
        summary = response.choices[0].message.content
        return summary.strip()

    def get_history(self, limit_turns: int = None, limit_chars: int = None, limit_words: int = None) -> List[Dict[str, str]]:
        hist = list(self.history)
        if limit_turns is not None:
            hist = hist[-limit_turns:]
        elif limit_words is not None:
            joined = ' '.join([m['text'] for m in hist])
            words = joined.split()
            if len(words) > limit_words:
                words = words[-limit_words:]
            hist = [{'role': 'assistant', 'text': ' '.join(words)}]
        elif limit_chars is not None:
            joined = ' '.join([m['text'] for m in hist])
            if len(joined) > limit_chars:
                joined = joined[-limit_chars:]
            hist = [{'role': 'assistant', 'text': joined}]
        return hist

    def show_history(self):
        for i, m in enumerate(self.history, 1):
            print(f"{i}. [{m['role']}] {m['text'][:200]}{'...' if len(m['text']) > 200 else ''}")

def demo_conversation_demo():
    cm = ConversationManager(summarize_after_k_runs=3)
    samples = [
        ('user', 'Hi, I want to book a flight to Mumbai next month.'),
        ('assistant', 'Sure — what dates are you considering and from which city?'),
        ('user', 'I am flexible but prefer early May; I am flying from Delhi.'),
        ('assistant', 'Got it — do you have any airline preference?'),
        ('user', 'No, I just want a cheap option. Also, I will need a vegetarian meal.'),
        ('assistant', 'Okay, I will look for the cheapest fares with veg meal option.'),
        ('user', 'Thanks! Also share hotel suggestions near Colaba, please.'),
        ('assistant', 'Will do — what is your budget per night?'),
    ]

    for role, text in samples:
        cm.add_message(role, text)
        print("\n-- After adding message --")
        cm.show_history()

    print("\n=== Final history (full) ===")
    cm.show_history()
    print("\n=== Demonstrate truncation by turns (last 3 turns) ===")
    for m in cm.get_history(limit_turns=3):
        print(m)
    print("\n=== Demonstrate truncation by chars (last 120 chars) ===")
    for m in cm.get_history(limit_chars=120):
        print(m)
    print("\n=== Demonstrate truncation by words (last 20 words) ===")
    for m in cm.get_history(limit_words=20):
        print(m)

if __name__ == "__main__":
    print("Running conversation demo with REAL Groq API using a supported model...\n")
    demo_conversation_demo()


Running conversation demo with REAL Groq API using a supported model...


-- After adding message --
1. [user] Hi, I want to book a flight to Mumbai next month.

-- After adding message --
1. [user] Hi, I want to book a flight to Mumbai next month.
2. [assistant] Sure — what dates are you considering and from which city?

-- After adding message --
1. [assistant] Conversation summary: Early May is a good time to fly to Mumbai, the weather is generally pleasant. 

As you're flying from Delhi, it's a relatively short domestic flight within India. I can suggest b...

-- After adding message --
1. [assistant] Conversation summary: Early May is a good time to fly to Mumbai, the weather is generally pleasant. 

As you're flying from Delhi, it's a relatively short domestic flight within India. I can suggest b...
2. [assistant] Got it — do you have any airline preference?

-- After adding message --
1. [assistant] Conversation summary: Early May is a good time to fly to Mumbai, the weather is 

In [25]:
import re
from pydantic import BaseModel, Field
from typing import Optional

class PersonSchema(BaseModel):
    name: str = Field(..., description="The person's full name")
    email: Optional[str] = Field(None, description="The person's email")
    phone: Optional[str] = Field(None, description="The person's phone number")
    location: Optional[str] = Field(None, description="The person's city or location")
    age: Optional[int] = Field(None, description="The person's age")

def extract_person_info(text: str) -> PersonSchema:
    # --- Name ---
    name_match = re.search(r"(?:my name is|I'm|I am|this is)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)", text, re.I)
    name = name_match.group(1).strip() if name_match else "Unknown"

    # --- Email ---
    email_match = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", text)
    email = email_match.group(0).rstrip(".") if email_match else None

    # --- Phone ---
    phone_match = re.search(r"(?:\+91)?\d{10}", text)
    phone = None
    if phone_match:
        phone = phone_match.group(0)
        if not phone.startswith("+91"):
            phone = "+91" + phone

    # --- Location ---
    loc_match = re.search(r"(?:in|from|Lives in|currently in)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)", text, re.I)
    location = loc_match.group(1).strip() if loc_match else None
    if location:
        location = re.sub(r"\b(and|currently|now)\b.*", "", location).strip()

    # --- Age ---
    age_match = re.search(r"\b(\d{1,2})(?=\s*(?:years old|yrs|yo|yrs old))", text, re.I)
    if not age_match:
        # fallback: catch "Age <number>"
        age_match = re.search(r"Age\s+(\d{1,2})", text, re.I)
    age = int(age_match.group(1)) if age_match else None

    return PersonSchema(name=name, email=email, phone=phone, location=location, age=age)

# --- Tests ---
if __name__ == "__main__":
    chats = [
        "Hi, my name is Anuradha Sharma. You can reach me at anuradha.sharma@example.com or +919876543210. I'm 24 years old and currently in Pune.",
        "Hello! I'm Rohan. rohan1995@mail.com. Phone: 9876543210. I'm from Mumbai and 28 years old.",
        "Hey there — this is Priya. priya.contact@mail.co.in. Lives in New Delhi. Age 30."
    ]
    for i, chat in enumerate(chats, 1):
        extracted = extract_person_info(chat)
        print(f"\n--- Chat #{i} ---")
        print(chat)
        print("Extracted:", extracted.model_dump())
        print("Validation: OK")



--- Chat #1 ---
Hi, my name is Anuradha Sharma. You can reach me at anuradha.sharma@example.com or +919876543210. I'm 24 years old and currently in Pune.
Extracted: {'name': 'Anuradha Sharma', 'email': 'anuradha.sharma@example.com', 'phone': '+919876543210', 'location': 'Pune', 'age': 24}
Validation: OK

--- Chat #2 ---
Hello! I'm Rohan. rohan1995@mail.com. Phone: 9876543210. I'm from Mumbai and 28 years old.
Extracted: {'name': 'Rohan', 'email': 'rohan1995@mail.com', 'phone': '+919876543210', 'location': 'Mumbai', 'age': 28}
Validation: OK

--- Chat #3 ---
Hey there — this is Priya. priya.contact@mail.co.in. Lives in New Delhi. Age 30.
Extracted: {'name': 'Priya', 'email': 'priya.contact@mail.co.in', 'phone': None, 'location': 'New Delhi', 'age': 30}
Validation: OK
